In [1]:
!pip install numpy pandas scikit-learn nltk spacy


   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.1 MB 495.5 kB/s eta 0:00:25
   ---------------------------------------- 0.1/12.1 MB 435.7 kB/s eta 0:00:28
   ---------------------------------------- 0.1/12.1 MB 393.8 kB/s eta 0:00:31
   ---------------------------------------- 0.1/12.1 MB 454.0 kB/s eta 0:00:27
   ---------------------------------------- 0.1/12.1 MB 463.7 kB/s eta 0:00:26
   ---------------------------------------- 0.1/12.1 MB 463.7 kB/s eta 0:00:26
    --------------------------------------- 0.2/12.1 MB 506.0 kB/s eta 0:00:24
    --------------------------------------- 0.2/12.1 MB 506.0 kB/s eta 0:00:24
    --------------------------------------- 0.2/12.1 MB 445.2 kB/s eta 0:00:27
    --------------------------------------- 0.2/12.1 MB 452.1 kB/s eta 0:00:27
    --------------------------------------- 0.3/12.1 MB 468.0 kB/s eta

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import spacy
spacy.cli.download('en_core_web_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mdsha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mdsha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [34]:
import pandas as pd

# Example data loading
resumes = pd.read_csv(r"C:\Users\mdsha\Downloads\archive (10)\UpdatedResumeDataSet.csv")  # resumes.csv should contain columns like 'id', 'resume_text'
job_descriptions = pd.read_csv(r"C:\Users\mdsha\Downloads\Data Scientist.csv")  # job_descriptions.csv should contain columns like 'id', 'job_description'

In [36]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = word_tokenize(text)
    text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words]
    return ' '.join(text)

resumes['cleaned_resume'] = resumes['Resume'].apply(preprocess_text)
job_descriptions['cleaned_job_description'] = job_descriptions['skills'].apply(preprocess_text)


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

resume_tfidf = vectorizer.fit_transform(resumes['cleaned_resume'])
job_description_tfidf = vectorizer.transform(job_descriptions['cleaned_job_description'])

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(resume_tfidf, job_description_tfidf)

In [41]:
def rank_resumes(similarity_scores):
    ranked_indices = similarity_scores.argsort()[::-1]
    return ranked_indices

job_description_id = 0  # Assuming we're matching against the first job description
resume_scores = similarity_scores[:, job_description_id]
ranked_resumes = rank_resumes(resume_scores)

In [42]:
resumes = pd.DataFrame({'id': range(1, 5), 'resume_text': example_resumes})
job_descriptions = pd.DataFrame({'id': [1], 'job_description': [example_job_description]})

In [43]:
ranked_resume_ids = resumes.iloc[ranked_resumes]['id']
ranked_scores = resume_scores[ranked_resumes]

for idx, score in zip(ranked_resume_ids, ranked_scores):
    print(f'Resume ID: {idx}, Relevance Score: {score}')

Resume ID: 1, Relevance Score: 0.8041384586865867
Resume ID: 4, Relevance Score: 0.5219627129665647
Resume ID: 3, Relevance Score: 0.3158721256993352
Resume ID: 2, Relevance Score: 0.0
